In [1]:
import re
import pandas as pd

chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\।]'

colnames = ['fpath', 'userid', 'text']
df = pd.read_csv('./content/asr_nepali/utt_spk_text.tsv', sep='\t', header=None, names=colnames)
# Folder structure for the files
df['path'] = './content/asr_nepali/data/' + df['fpath'].str[:2]
# Add file link to the path
df['path'] = df['path'] + "/" + df['fpath'] + '.flac'

In [ ]:
df

In [2]:
df = df.drop_duplicates(subset=['file'])

In [3]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.1)

In [4]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

In [5]:
train_dataset

Dataset({
    features: ['file', 'phrase', 'path', '__index_level_0__'],
    num_rows: 1838
})

In [6]:
test_dataset

Dataset({
    features: ['file', 'phrase', 'path', '__index_level_0__'],
    num_rows: 205
})

In [7]:
test_dataset = test_dataset.remove_columns(['file'])
train_dataset = train_dataset.remove_columns(['file'])

In [8]:
def remove_special_characters(batch):
    batch["phrase"] = re.sub(chars_to_ignore_regex, '', batch["phrase"]).lower() + " "
    return batch

In [9]:
test_dataset = test_dataset.map(remove_special_characters)
train_dataset = train_dataset.map(remove_special_characters)

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
test_dataset = test_dataset.map(extract_all_chars remove_columns=test_dataset.column_names)
train_dataset = train_dataset.map(extract_all_chars, remove_columns=train_dataset.column_names)

In [ ]:
vocab_list = list(set(test_dataset["vocab"][0]) | set(train_dataset["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}

In [ ]:
try:
  del vocab_dict['\\']
  del vocab_dict['a']
  del vocab_dict['b']
  del vocab_dict['c']
  del vocab_dict['e']
  del vocab_dict['f']
  del vocab_dict['k']
  del vocab_dict['o']
  del vocab_dict['\xa0']
  del vocab_dict['\u200c']
  del vocab_dict['\u200d']
  del vocab_dict['\u200e']
  del vocab_dict['\u200f']
except:
  pass

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

In [10]:
from transformers import Wav2Vec2CTCTokenizer,Wav2Vec2FeatureExtractor,Wav2Vec2Processor

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True,
                                             return_attention_mask=True)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [11]:
import torchaudio
import random
import gc

resampler = torchaudio.transforms.Resample(48_000, 16_000)

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    batch["sampling_rate"] = 16000
    batch["target_text"] = batch["phrase"]
    if random.random() < 0.25:
        gc.collect()
    return batch

d:\work\asr\transformer_test\env\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


In [12]:
train_dataset = train_dataset.map(speech_file_to_array_fn)
test_dataset = test_dataset.map(speech_file_to_array_fn)

In [13]:
train_dataset

Dataset({
    features: ['__index_level_0__', 'path', 'phrase', 'sampling_rate', 'speech', 'target_text'],
    num_rows: 1838
})

In [14]:
def prepare_dataset(batch):
    batch["input_values"] = processor(batch["speech"], sampling_rate=16000).input_values[0]

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [15]:
train_dataset = train_dataset.map(prepare_dataset, remove_columns = train_dataset.column_names)
test_dataset = test_dataset.map(prepare_dataset,remove_columns = test_dataset.column_names)

In [16]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
import torch

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [17]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [18]:
from datasets import load_metric

wer_metric = load_metric("wer")

In [19]:
import numpy as np

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [26]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "./models/wav2vec2-xlsr-nepali",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    torch.device('cpu')
model.to(device)
model.freeze_feature_extractor()

In [27]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./content/transfer-learning-test-2",
    # group_by_length=True,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=10,
    fp16=True,
    save_steps=500,
    eval_steps=100,
    logging_steps=100,
    learning_rate=3e-4,
    warmup_steps=200,
    save_total_limit=1,
    eval_accumulation_steps=1
)

In [28]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
)

In [29]:
trainer.train(True)
trainer.save_model('content/transfer-learning-test-2')
trainer.tokenizer.save_pretrained('content/transfer-learning-test-2')
processor.save_pretrained('content/transfer-learning-test-2')

ValueError: No valid checkpoint found in output directory (./content/transfer-learning-test-2)